In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import tikzplotlib

try:
    sys.path.append(str(Path().cwd().parent))
except IndexError:
    pass

from external.bjontegaard_metrics.bj_delta import *

In [ ]:
BASE_PATH = Path.cwd() / "bd-roi"
GOP = 1

all_files = list(BASE_PATH.glob("*.csv"))

df = pd.concat([pd.read_csv(data_file, index_col="name") for data_file in all_files])

In [ ]:
gop = df.loc[df.gop_len == GOP]

base = gop.loc[gop["mode"].str.contains("base")]
cont = gop.loc[gop["mode"].str.contains("roi_cont")]
block = gop.loc[gop["mode"].str.contains("roi-")]

base_per_qp = base.groupby(["qp"]).mean()
cont_per_qp = cont.groupby(["qp"]).mean()
block_per_qp = block.groupby(["qp"]).mean()

In [ ]:
def print_bj_delta(df_one, df_two):
    print(f"BD-PSNR: {bj_delta(df_one.bitrate, df_one.psnr, df_two.bitrate, df_two.psnr, mode=0):.2f}")
    print(f"BD-RATE: {bj_delta(df_one.bitrate, df_one.psnr, df_two.bitrate, df_two.psnr, mode=1):.2f}")

    print(f"BD-VMAF: {bj_delta(df_one.bitrate, df_one.vmaf, df_two.bitrate, df_two.vmaf, mode=0):.2f}")
    print(f"BD-RATE: {bj_delta(df_one.bitrate, df_one.vmaf, df_two.bitrate, df_two.vmaf, mode=1):.2f}")

def plot_rd_curve(metric: str):
    plt.scatter(base_per_qp.bitrate, base_per_qp[metric], label="Baseline")
    plt.plot(base_per_qp.bitrate, base_per_qp[metric])
    plt.scatter(cont_per_qp.bitrate, cont_per_qp[metric], label="Continuous")
    plt.plot(cont_per_qp.bitrate, cont_per_qp[metric])
    plt.scatter(block_per_qp.bitrate, block_per_qp[metric], label="Block-based")
    plt.plot(block_per_qp.bitrate, block_per_qp[metric])
    plt.xlabel("Bitrate")
    plt.ylabel(metric.upper())
    plt.legend(loc="lower right")
    tikzplotlib.save(f"roi_rd_{metric}_{GOP}.tex")

In [ ]:
plot_rd_curve("psnr")

In [ ]:
plot_rd_curve("vmaf")

In [ ]:
print("Block-based mode")
print_bj_delta(base, cont)
print()
print("Continuous mode")
print_bj_delta(base, block)